# Kaggle Spacehip Titanic

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
%cd /content/drive/MyDrive/"Colab Notebooks"/"Kaggle Spaceship Titanic"

/content/drive/MyDrive/Colab Notebooks/Kaggle Spaceship Titanic


In [10]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [22]:
!git status

On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)

	new file:   sample_submission.csv
	new file:   spaceship_titanic.ipynb
	new file:   test.csv
	new file:   train.csv



In [25]:
!git commit -m "First commit"

[master (root-commit) 0e00f36] First commit
 4 files changed, 17251 insertions(+)
 create mode 100644 sample_submission.csv
 create mode 100644 spaceship_titanic.ipynb
 create mode 100644 test.csv
 create mode 100644 train.csv


In [24]:
!git config --global user.email "teamchrisN@gmail.com"
!git config --global user.name "Christopher Nguyen"